In [80]:
import config
import spotipy
import pandas as pd
import json
import pprint
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from sklearn.cluster import KMeans


#Initialize SpotiPy with user credentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))



In [82]:
#LOAD TRAINED MODEL

import pickle

# Load the model
with open('spotify_kmeans_model.pkl', 'rb') as file:
    kmeans_model = pickle.load(file)

In [85]:
#LOAD DATABASE

df5k2 = pd.read_csv("df5k2.csv")
df5k2

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,id,names,id.1,names.1
0,0.513,0.868,-5.061,0.0410,0.01850,0.000000,0.1210,0.3670,92.009,230467,1,2HYFX63wP3otVIvopRS99Z,Houdini,2HYFX63wP3otVIvopRS99Z,Houdini
1,0.646,0.359,-10.388,0.1550,0.24700,0.914000,0.0872,0.1540,119.885,360147,1,7lQ8MOhq6IN2w8EYcFNSUk,Without Me,7lQ8MOhq6IN2w8EYcFNSUk,Without Me
2,0.825,0.652,-3.183,0.0802,0.58100,0.000000,0.0931,0.9310,95.977,233713,1,561jH07mF1jHuk7KlaeF0s,Mockingbird,561jH07mF1jHuk7KlaeF0s,Mockingbird
3,0.694,0.770,-5.335,0.1490,0.17600,0.000011,0.1180,0.1630,125.905,184560,1,4xkOaSrkexMciUUogZKVTS,Till I Collapse,4xkOaSrkexMciUUogZKVTS,Till I Collapse
4,0.730,0.769,-5.114,0.0298,0.13800,0.000000,0.0473,0.5070,104.948,228093,1,3yfqSUWxFvZELEM4PmlwIR,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,The Real Slim Shady
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4630,0.719,0.580,-6.173,0.0903,0.08180,0.812000,0.1170,0.4900,170.099,131294,1,NaN,NaN,NaN,NaN
4631,0.449,0.585,-6.761,0.0268,0.05090,0.000087,0.0833,0.2120,139.631,278719,1,NaN,NaN,NaN,NaN
4632,0.690,0.723,-9.204,0.0639,0.17400,0.000000,0.2550,0.4870,149.987,157192,1,NaN,NaN,NaN,NaN
4633,0.626,0.674,-10.002,0.0411,0.00237,0.863000,0.1060,0.0489,124.977,375361,1,NaN,NaN,NaN,NaN


In [88]:

song_name = input("Write the song:") #GET USER INPUT

Write the song: love


In [90]:
def bring_song (song_name):
    result = sp.search(q=song_name, limit=1, market ="ES") #SEARCH FOR THE RESULT
    id = result["tracks"]["items"][0]["id"] #BRING THE ID
    return id

In [92]:
def classify_song (id):
    features = sp.audio_features(id) #BRING ALL FEATURES
    X=pd.DataFrame(features) #TURN ARRAY INTO DATAFRAME
    X=X[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]] #CHANGE THE COLUMN NAMES
    X = X.drop("id", axis=1) #DROP ID
    cluster = kmeans_model.predict(X) #RUN PREDICTION 
    cluster = cluster[0] #CHANGE ARRAY TO NUMBER
    return cluster

In [94]:
def song_recomender (cluster):
    same_cluster_songs = df.loc[df['cluster'] == cluster] #SEARCH FOR ALL THE SONGS WITH THE SAME CLUSTER
    random_sample = same_cluster_songs.sample(n=1) #SELECT RANDOMLY 1 SONG 
    return random_sample

In [96]:
song_id = bring_song (song_name)

In [98]:
cluster = classify_song(song_id)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=3wm27SCXXbKtLVv1B6JLFN:
 Max Retries, reason: too many 429 error responses

In [ ]:
song = song_recomender(cluster)
song